In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split



Saving heart.csv to heart (2).csv


In [ ]:

df = pd.read_csv('heart.csv')
getter = df['output']
df = df.drop(columns=['output'])
print(df.head())
print('output check')
print(getter.head())



   age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0   63    1   3     145   233    1        0       150     0      2.3    0   
1   37    1   2     130   250    0        1       187     0      3.5    0   
2   41    0   1     130   204    0        0       172     0      1.4    2   
3   56    1   1     120   236    0        1       178     0      0.8    2   
4   57    0   0     120   354    0        1       163     1      0.6    2   

   caa  thall  
0    0      1  
1    0      2  
2    0      2  
3    0      2  
4    0      2  
output check
0    1
1    1
2    1
3    1
4    1
Name: output, dtype: int64


Check to make sure DataFile Heart.csv works well i.e Has all data besides output

In [ ]:
class Model(nn.Module):
  def __init__(self, inputs = 13, hiddenlayer1 = 64, hiddenlayer2 = 32, hiddenlayer3 = 16, outputs = 2):
    super().__init__()
    self.fc1 = nn.Linear(inputs, hiddenlayer1)
    self.fc2 = nn.Linear(hiddenlayer1, hiddenlayer2)
    self.fc3 = nn.Linear(hiddenlayer2, hiddenlayer3)
    #self.fc4 = nn.Linear(hiddenlayer3, hiddenlayer4)
    self.output = nn.Linear(hiddenlayer3, outputs)
    self.dropout = nn.Dropout(0.2)



  def forwardfeed(self, x):

    x = F.relu(self.fc1(x))

    x = self.dropout(x)
    x = F.relu(self.fc2(x))

    x = self.dropout(x)
    x = F.relu(self.fc3(x))

    x = self.dropout(x)
    #x = F.relu(self.fc4(x))
    x = self.output(x)
    return x

torch.manual_seed(1)
myModel = Model()

input = df.values
output = getter.values


input_train, input_test, output_train, output_test = train_test_split(input, output, test_size=0.1, random_state=1)

input_train = torch.FloatTensor(input_train)
input_test = torch.FloatTensor(input_test)

output_train = torch.LongTensor(output_train)
output_test = torch.LongTensor(output_test)




error = nn.CrossEntropyLoss()
optimization = torch.optim.Adam(myModel.parameters(), lr=1e-3, weight_decay=1e-4)


#optimization = torch.optim.SGD(myModel.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)





In [ ]:

epochs = 1000
losses = []
for i in range(epochs):
        i = i+1
        y_pred = myModel.forwardfeed(input_train)
        loss = error(y_pred, output_train)
        losses.append(loss)

        if i % 200 == 0:
            print(f'epoch: {i} loss: {loss}')
        optimization.zero_grad()
        loss.backward()
        optimization.step()

        #torch.nn.utils.clip_grad_norm_(myModel.parameters(), max_norm=1.0)


epoch: 200 loss: 0.46735888719558716
epoch: 400 loss: 0.4685838520526886
epoch: 600 loss: 0.3891963064670563
epoch: 800 loss: 0.31981417536735535
epoch: 1000 loss: 0.30973270535469055


In [ ]:
with torch.no_grad():
    y_eval = myModel.forwardfeed(input_test)
    loss = error(y_eval, output_test)
    print(loss)

tensor(0.3319)


In [ ]:
correct = 0;
total = 0;

with torch.no_grad():

   for i, data in enumerate(input_test):
       total += 1
       y_val = myModel.forwardfeed(data)

       s = '';
       if output_test[i] == 1:
           s = '1'
       else:
           s = '0'

       if y_val.argmax().item() == output_test[i]:
           correct += 1

       print(f'{y_val.argmax().item()}, {s}')

print((correct/total)*100)

0, 0
1, 1
0, 0
0, 0
0, 0
0, 0
1, 0
1, 1
0, 0
1, 1
1, 1
0, 0
1, 0
1, 0
0, 0
1, 0
1, 1
0, 0
1, 1
1, 0
1, 1
0, 0
1, 1
1, 1
1, 1
1, 1
0, 0
1, 1
1, 1
1, 1
1, 1
83.87096774193549
